<a href="https://colab.research.google.com/github/DarriusChen/Regulatory-Compliance-Mapping-Database/blob/main/flatten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Flattening function (can customize)

In [71]:
def flatten(filePath,columns,flatColumn,splitBy,export,outputFileName):
  df = pd.read_csv(filePath,header=None)
  df.columns = columns
  df[flatColumn] = df[flatColumn].str.strip()
  df[flatColumn] = df[flatColumn].str.split(splitBy)
  df = df.explode(flatColumn).reset_index(drop=True)
  df[flatColumn] = df[flatColumn].str.strip()

  # check if it contains null value
  if df.isnull().values.any():
    print("The DataFrame contains null values.")
    rows_with_null = df[df.isnull().any(axis=1)]
    return rows_with_null
  else:
    if export==True:
      df.to_csv(outputFileName, index=False)
    else:
      return df

# CSF table

### 初始處理

In [4]:
csf_df  = pd.read_csv("/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/CSF_sub.csv",header=None)
csf_df.head()

,0,1,2,3,4,5
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,"CM-8, PM-5"
1,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
2,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,"AC-4, CA-3, CA-9, PL-8, SA-17"
3,ID.AM-4,External information systems are catalogued,V1.1,ID,ID.AM,"AC-20, PM-5, SA-9"
4,ID.AM-5,"Resources (e.g., hardware, devices, data, time...",V1.1,ID,ID.AM,"CP-2, RA-2, RA-9, SA-20, SC-6"


In [5]:
csf_df.columns = ["csf_s_ID","csf_s_content","csf_s_version","csf_f_ID","csf_c_ID","sp80053_c_ID"]
csf_df

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,"CM-8, PM-5"
1,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
2,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,"AC-4, CA-3, CA-9, PL-8, SA-17"
3,ID.AM-4,External information systems are catalogued,V1.1,ID,ID.AM,"AC-20, PM-5, SA-9"
4,ID.AM-5,"Resources (e.g., hardware, devices, data, time...",V1.1,ID,ID.AM,"CP-2, RA-2, RA-9, SA-20, SC-6"
...,...,...,...,...,...,...
103,RC.IM-1,Recovery plans incorporate lessons learned,V1.1,RC,RC.IM,"CP-2, IR-4, IR-8"
104,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,"CP-2, IR-4, IR-8"
105,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
106,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4


In [110]:
type(csf_df['sp80053_c_ID'][2])

str

### 第一階段unstack & flatten

In [6]:
# Split and stack the 'sp80053_c_ID' column
csf_df['sp80053_c_ID'] = csf_df['sp80053_c_ID'].str.split(', ')
df = csf_df.explode('sp80053_c_ID').reset_index(drop=True)

# Print the flattened DataFrame
df

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
493,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
494,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
495,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
496,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


In [7]:
def flatten_csf(filePath,columns,flatColumn,outputFileName):
  df = pd.read_csv(filePath,header=None)
  df.columns = columns
  df[flatColumn] = df[flatColumn].str.split(', ')
  df = df.explode(flatColumn).reset_index(drop=True)

  # check if it contains null value
  if df.isnull().values.any():
    print("The DataFrame contains null values.")
    rows_with_null = df[df.isnull().any(axis=1)]
    return rows_with_null
  else:
    # df.to_csv(outputFileName, index=False)
    return df

In [9]:
flatten_csf(filePath="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/CSF_sub.csv",\
        columns=["csf_s_ID","csf_s_content","csf_s_version","csf_f_ID","csf_c_ID","sp80053_c_ID"],\
        flatColumn="sp80053_c_ID",\
        outputFileName="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/csf_subcat_new.csv")

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
493,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
494,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
495,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
496,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


In [11]:
csf_v1 = flatten_csf(filePath="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/CSF_sub.csv",\
        columns=["csf_s_ID","csf_s_content","csf_s_version","csf_f_ID","csf_c_ID","sp80053_c_ID"],\
        flatColumn="sp80053_c_ID",\
        outputFileName="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/csf_subcat_new.csv")
csf_v1

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
493,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
494,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
495,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
496,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


### Read 所有-1的CSF category table，將-1 controls...取代為所有-1組成的list

In [1]:
# all_one = [(i+"-1") for i in list(all_df.iloc[:,0])]
all_one = [
    'AC-1',
    'AT-1',
    'AU-1',
    'CA-1',
    'CM-1',
    'CP-1',
    'IA-1',
    'IR-1',
    'MA-1',
    'MP-1',
    'PE-1',
    'PL-1',
    'PM-1',
    'PS-1',
    'PT-1',
    'RA-1',
    'SA-1',
    'SC-1',
    'SI-1',
    'SR-1'
]
all_one

['AC-1',
 'AT-1',
 'AU-1',
 'CA-1',
 'CM-1',
 'CP-1',
 'IA-1',
 'IR-1',
 'MA-1',
 'MP-1',
 'PE-1',
 'PL-1',
 'PM-1',
 'PS-1',
 'PT-1',
 'RA-1',
 'SA-1',
 'SC-1',
 'SI-1',
 'SR-1']

In [12]:
# Define the specific string you want to check for
specific_string = "-1 controls"

# Define a custom function to apply the replacement
def replace_with_allOne(value):
    if specific_string in value:
        return all_one
    else:
        return value
csf_v2 = csf_v1
# Apply the custom function to the "sp80053_c_ID" column
csf_v2['sp80053_c_ID'] = csf_v1['sp80053_c_ID'].apply(replace_with_allOne)
csf_v2

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
493,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
494,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
495,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
496,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


### 第二階段 Unstack & flatten

In [13]:
csf_v3 = csf_v2.explode("sp80053_c_ID").reset_index(drop=True)
csf_v3

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
550,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
551,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
552,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
553,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


### Export

In [14]:
csf_v3.to_csv(path_or_buf="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/csf_subcat_final.csv",\
              index=False)

# SP80053-ISO Mapping

In [111]:
spiso_df = pd.read_csv("/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/SP80053_ISO_mapping.csv")
spiso_df

,NIST SP 800-53 REVISION 5,CONTROL NAMES,ISO/IEC 27001:2022 REQUIREMENTS AND CONTROLS
0,AC-1,Access Control Policy and Procedures,"5.2, 5.3, 7.5.1, 7.5.2, 7.5.3, A.5.1, A.5.2, A..."
1,AC-2,Account Management,"A.5.16, A.5.18, A.8.2"
2,AC-3,Access Enforcement,"A.5.15, A.5.33, A.8.3, A.8.4, A.8.18, A.8.20, ..."
3,AC-4,Information Flow Enforcement,"A.5.14, A.8.22, A.8.23"
4,AC-5,Separation of Duties,A.5.3
...,...,...,...
164,SR-3,Supply Chain Controls and Processes,"A.5.20, A.5.21"
165,SR-4,Provenance,"A.5.21, A.8.30"
166,SR-5,"Acquisition Strategies, Tools, and Methods","A.5.20, A.5.21, A.5.23"
167,SR-6,Supplier Assessments and Reviews,A.5.22


In [120]:
spiso_new = flatten(filePath="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/SP80053_ISO_mapping.csv",\
                    flatColumn="ISO/IEC 27001:2022 REQUIREMENTS AND CONTROLS",\
                    columns=spiso_df.columns,
                    splitBy=', ',\
                    export=False,\
                    outputFileName=None)
spiso_new

,NIST SP 800-53 REVISION 5,CONTROL NAMES,ISO/IEC 27001:2022 REQUIREMENTS AND CONTROLS
0,AC-1,Access Control Policy and Procedures,5.2
1,AC-1,Access Control Policy and Procedures,5.3
2,AC-1,Access Control Policy and Procedures,7.5.1
3,AC-1,Access Control Policy and Procedures,7.5.2
4,AC-1,Access Control Policy and Procedures,7.5.3
...,...,...,...
617,SR-5,"Acquisition Strategies, Tools, and Methods",A.5.20
618,SR-5,"Acquisition Strategies, Tools, and Methods",A.5.21
619,SR-5,"Acquisition Strategies, Tools, and Methods",A.5.23
620,SR-6,Supplier Assessments and Reviews,A.5.22


In [124]:
spiso_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622 entries, 0 to 621
Data columns (total 3 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   NIST SP 800-53 REVISION 5                     622 non-null    object
 1   CONTROL NAMES                                 622 non-null    object
 2   ISO/IEC 27001:2022 REQUIREMENTS AND CONTROLS  622 non-null    object
dtypes: object(3)
memory usage: 14.7+ KB


In [125]:
spiso_new.to_csv(path_or_buf="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/sp80053_iso_mapping_new.csv",\
              index=False)

# Attack-Defend Mapping

In [66]:
att_df = pd.read_csv('/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/need_flatten.csv')
att_df

,T1548,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed to control elevate privileges to gain higher-level permissions. Most modern systems contain native elevation control mechanisms that are intended to limit privileges that a user can perform on a machine. Authorization has to be granted to specific users in order to perform tasks that can be considered of higher risk. An adversary can perform several methods to take advantage of built-in control mechanisms in order to escalate privileges on a system.,"Monitor the file system for files that have the setuid or setgid bits set. Also look for any process API calls for behavior that may be indicative of [Process Injection](https://attack.mitre.org/techniques/T1055) and unusual loaded DLLs through [DLL Search Order Hijacking](https://attack.mitre.org/techniques/T1574/001), which indicate attempts to gain access to higher privileged processes. On Linux, auditd can alert every time a user's actual ID and effective ID are different (this is what happens when you sudo).\n\nConsider monitoring for <code>/usr/libexec/security_authtrampoline</code> executions which may indicate that AuthorizationExecuteWithPrivileges is being executed. MacOS system logs may also indicate when AuthorizationExecuteWithPrivileges is being called. Monitoring OS API callbacks for the execution can also be a way to detect this behavior but requires specialized security tooling.\n\nOn Linux, auditd can alert every time a user's actual ID and effective ID are different (this is what happens when you sudo). This technique is abusing normal functionality in macOS and Linux systems, but sudo has the ability to log all input and output based on the <code>LOG_INPUT</code> and <code>LOG_OUTPUT</code> directives in the <code>/etc/sudoers</code> file.\n\nThere are many ways to perform UAC bypasses when a user is in the local administrator group on a system, so it may be difficult to target detection on all variations. Efforts should likely be placed on mitigation and collecting enough information on process launches and actions that could be performed before and after a UAC bypass is performed. Some UAC bypass methods rely on modifying specific, user-accessible Registry settings. Analysts should monitor Registry settings for unauthorized changes.",technique(level 2),V13,"Defense Evasion, Privilege Escalation",Unnamed: 7
0,T1548.002,Abuse Elevation Control Mechanism: Bypass User...,Adversaries may bypass UAC mechanisms to eleva...,There are many ways to perform UAC bypasses wh...,subtechnique(level 3),V13,"Defense Evasion, Privilege Escalation",NaN
1,T1548.004,Abuse Elevation Control Mechanism: Elevated Ex...,Adversaries may leverage the <code>Authorizati...,Consider monitoring for <code>/usr/libexec/sec...,subtechnique(level 3),V13,"Defense Evasion, Privilege Escalation",NaN
2,T1548.001,Abuse Elevation Control Mechanism: Setuid and ...,An adversary may abuse configurations where an...,Monitor the file system for files that have th...,subtechnique(level 3),V13,"Defense Evasion, Privilege Escalation",NaN
3,T1548.003,Abuse Elevation Control Mechanism: Sudo and Su...,Adversaries may perform sudo caching and/or us...,"On Linux, auditd can alert every time a user's...",subtechnique(level 3),V13,"Defense Evasion, Privilege Escalation",NaN
4,T1134,Access Token Manipulation,Adversaries may modify access tokens to operat...,If an adversary is using a standard command-li...,technique(level 2),V13,"Defense Evasion, Privilege Escalation",NaN
...,...,...,...,...,...,...,...,...
601,T1102.002,Web Service: Bidirectional Communication,"Adversaries may use an existing, legitimate ex...",Host data that can relate unknown or suspiciou...,subtechnique(level 3),V13,Command and Control,NaN
602,T1102.001,Web Service: Dead Drop Resolver,"Adversaries may use an existing, legitimate ex...",Host data that can relate unknown or suspiciou...,subtechnique(level 3),V13,Command and Control,NaN
603,T1102.003,Web Service

In [72]:
att_df_v1 = flatten(filePath="/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/need_flatten.csv",\
        columns=['attack_t_ID','attack_t_name','attack_t_desc','attack_t_detection','attack_t_label','attack_t_version','attack_tac_name','haha'],\
        flatColumn="attack_tac_name",\
        splitBy=", ",\
        export=False,\
        outputFileName=None)
att_df_v1.info()

The DataFrame contains null values.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 785 entries, 0 to 784
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   attack_t_ID         785 non-null    object 
 1   attack_t_name       785 non-null    object 
 2   attack_t_desc       785 non-null    object 
 3   attack_t_detection  750 non-null    object 
 4   attack_t_label      785 non-null    object 
 5   attack_t_version    785 non-null    object 
 6   attack_tac_name     785 non-null    object 
 7   haha                0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 55.2+ KB


In [73]:
att_df_v1

,attack_t_ID,attack_t_name,attack_t_desc,attack_t_detection,attack_t_label,attack_t_version,attack_tac_name,haha
0,T1548,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Monitor the file system for files that have th...,technique(level 2),V13,Defense Evasion,NaN
1,T1548,Abuse Elevation Control Mechanism,Adversaries may circumvent mechanisms designed...,Monitor the file system for files that have th...,technique(level 2),V13,Privilege Escalation,NaN
2,T1548.002,Abuse Elevation Control Mechanism: Bypass User...,Adversaries may bypass UAC mechanisms to eleva...,There are many ways to perform UAC bypasses wh...,subtechnique(level 3),V13,Defense Evasion,NaN
3,T1548.002,Abuse Elevation Control Mechanism: Bypass User...,Adversaries may bypass UAC mechanisms to eleva...,There are many ways to perform UAC bypasses wh...,subtechnique(level 3),V13,Privilege Escalation,NaN
4,T1548.004,Abuse Elevation Control Mechanism: Elevated Ex...,Adversaries may leverage the <code>Authorizati...,Consider monitoring for <code>/usr/libexec/sec...,subtechnique(level 3),V13,Defense Evasion,NaN
...,...,...,...,...,...,...,...,...
780,T1102.002,Web Service: Bidirectional Communication,"Adversaries may use an existing, legitimate ex...",Host data that can relate unknown or suspiciou...,subtechnique(level 3),V13,Command and Control,NaN
781,T1102.001,Web Service: Dead Drop Resolver,"Adversaries may use an existing, legitimate ex...",Host data that can relate unknown or suspiciou...,subtechnique(level 3),V13,Command and Control,NaN
782,T1102.003,Web Service: One-Way Communication,"Adversaries may use an existing, legitimate ex...",Host data that can relate unknown or suspiciou...,subtechnique(level 3),V13,Command and Control,NaN
783,T1047,Windows Management Instrumentation,Adversaries may abuse Windows Management Instr...,Monitor network traffic for WMI connections; t...,technique(level 2),V13,Execution,NaN


In [42]:
att_df_v1.to_csv("/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/flattened/attack_temp.csv", index=False)

# Debugging

In [15]:
csf_853_df = pd.read_csv("/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/flattening/flattened/csf_subcat_final.csv")
sp853_df = pd.read_csv("/content/drive/MyDrive/Regulatory_Compliance_Mapping_Database/code/debugging/temp1.csv")

In [16]:
csf_853_df

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,CM-8
1,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5
2,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3
...,...,...,...,...,...,...
550,RC.IM-2,Recovery strategies are updated,V1.1,RC,RC.IM,IR-8
551,RC.CO-1,Public relations are managed,V1.1,RC,RC.CO,IR-4
552,RC.CO-2,Reputation is repaired after an incident,V1.1,RC,RC.CO,IR-4
553,RC.CO-3,Recovery activities are communicated to intern...,V1.1,RC,RC.CO,CP-2


import numpy as np
from google.colab import autoviz

def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(csf_853_df, *['csf_f_ID'], **{})
chart

In [17]:
sp853_df

,ID.AM-1,Physical devices and systems within the organization are inventoried,V1.1,ID,ID.AM,CM-8,CM-8.1,control,System Component Inventory,a. Develop and document an inventory of system components that:\n1. Accurately reflects the system;\n2. Includes all components within the system;\n3. Does not include duplicate accounting of components or components assigned to any other system;\n4. Is at the level of granularity deemed necessary for tracking and reporting; and\n5. Includes the following information to achieve system component accountability: [Assignment: organization-defined information deemed necessary to achieve effective system component accountability]; and\nb. Review and update the system component inventory [Assignment: organization-defined frequency].,"System components are discrete, identifiable information technology assets that include hardware, software, and firmware. Organizations may choose to implement centralized system component inventories that include components from all organizational systems. In such situations, organizations ensure that the inventories include system-specific information required for component accountability. The information necessary for effective accountability of system components includes the system name, software owners, software version numbers, hardware inventory specifications, software license information, and for networked components, the machine names and network addresses across all implemented protocols (e.g., IPv4, IPv6). Inventory specifications include date of receipt, cost, model, serial number, manufacturer, supplier information, component type, and physical location.\nPreventing duplicate accounting of system components addresses the lack of accountability that occurs when component ownership and system association is not known, especially in large or complex connected systems. Effective prevention of duplicate accounting of system components necessitates use of a unique identifier for each component. For software inventory, centrally managed software that is accessed via other systems is addressed as a component of the system on which it is installed and managed. Software installed on multiple organizational systems and managed at the system level is addressed for each individual system and may appear more than once in a centralized component inventory, necessitating a system association for each software instance in the centralized inventory to avoid duplicate accounting of components. Scanning systems implementing multiple network protocols (e.g., IPv4 and IPv6) can result in duplicate components being identified in different address spaces. The implementation of CM-8(7) can help to eliminate duplicate accounting of components.","CM-2, CM-7, CM-9, CM-10, CM-11, CM-13, CP-2, CP-9, MA-2, MA-6, PE-20, PL-9, PM-5, SA-4, SA-5, SI-2, SR-4",Rev.5
0,ID.AM-1,Physical devices and systems within the organi...,V1.1,ID,ID.AM,PM-5,PM-5,control,System Inventory,Develop and update [Assignment: organization-d...,[OMB A-130] provides guidance on developing sy...,NaN,Rev.5
1,ID.AM-2,Software platforms and applications within the...,V1.1,ID,ID.AM,CM-8,CM-8,control,System Component Inventory,a. Develop and document an inventory of system...,"System components are discrete, identifiable i...","CM-2, CM-7, CM-9, CM-10, CM-11, CM-13, CP-2, C...",Rev.5
2,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,AC-4,AC-4,control,Information Flow Enforcement,Enforce approved authorizations for controllin...,Information flow control regulates where infor...,"AC-3, AC-6, AC-16, AC-17, AC-19, AC-21, AU-10,...",Rev.5
3,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-3,CA-3,control,Information Exchange,a. Approve and manage the exchange of informat...,System information exchange requirements apply...,"AC-4, AC-20, AU-16, CA-6, IA-3, IR-4, PL-2, PT...",Rev.5
4,ID.AM-3,Organizational communication and data flows ar...,V1.1,ID,ID.AM,CA-9,CA-9,control,Inter

In [18]:
csf_853_df.sp80053_c_ID.info()

<class 'pandas.core.series.Series'>
RangeIndex: 555 entries, 0 to 554
Series name: sp80053_c_ID
Non-Null Count  Dtype 
--------------  ----- 
555 non-null    object
dtypes: object(1)
memory usage: 4.5+ KB


In [19]:
sp853_df['CM-8'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 548 entries, 0 to 547
Series name: CM-8
Non-Null Count  Dtype 
--------------  ----- 
548 non-null    object
dtypes: object(1)
memory usage: 4.4+ KB


In [22]:
csf_853_df[~csf_853_df['sp80053_c_ID'].isin(sp853_df['CM-8'])]

,csf_s_ID,csf_s_content,csf_s_version,csf_f_ID,csf_c_ID,sp80053_c_ID
88,ID.GV-4,Governance and risk management processes addre...,V1.1,ID,ID.GV,RA-1
122,ID.RA-5,"Threats, vulnerabilities, likelihoods, and imp...",V1.1,ID,ID.RA,PM-16
157,ID.SC-4,Suppliers and third-party partners are routine...,V1.1,ID,ID.SC,CA-2
296,PR.IP-7,Protection processes are improved,V1.1,PR,PR.IP,IR-8
366,PR.PT-4,Communications and control networks are protected,V1.1,PR,PR.PT,SC-37
476,DE.DP-4,Event detection information is communicated,V1.1,DE,DE.DP,RA-5
